In [ ]:
import json
import numpy as np
from PIL import Image
from pathlib import Path
import cv2
from tqdm import tqdm

HUMAN_CLASSES = {24, 25}
VEHICLE_CLASSES = {26, 27, 28, 31, 32, 33}  

class CityscapesPreprocessor:
    def __init__(self, input_root, output_root, scale_factor=0.25):
        self.input_root = Path(input_root)
        self.output_root = Path(output_root)
        self.scale_factor = scale_factor
        
    def process_all_splits(self):
        for split in ['train', 'val', 'test']:
            print(f"\nProcessing {split} split...")
            self.process_split(split)
    
    def process_split(self, split):
        img_dir = self.input_root / 'leftImg8bit' / split
        gt_dir = self.input_root / 'gtFine' / split
        
        out_img_dir = self.output_root / split / 'images'
        out_mask_dir = self.output_root / split / 'masks'
        out_img_dir.mkdir(parents=True, exist_ok=True)
        out_mask_dir.mkdir(parents=True, exist_ok=True)
        
        image_files = []

        for city_dir in sorted(img_dir.iterdir()):
            if not city_dir.is_dir():
                continue

            for img_file in sorted(city_dir.glob('*_leftImg8bit.png')):
                base = img_file.stem.replace('_leftImg8bit', '')
                gt_city = gt_dir / city_dir.name
                instance_file = gt_city / f"{base}_gtFine_instanceIds.png"

                if instance_file.exists():
                    image_files.append((img_file, instance_file, base))

        saved_count = 0
        failed_count = 0
        for img_path, inst_path, base_name in tqdm(image_files, desc=f"{split}"):
            out_img_path = out_img_dir / f"{base_name}.png"
            out_mask_path = out_mask_dir / f"{base_name}.npy"
            
            if self._process_single_image(
                img_path, inst_path, base_name, out_img_dir, out_mask_dir
            ):
                if out_img_path.exists() and out_mask_path.exists():
                    try:
                        Image.open(out_img_path).load()
                        np.load(out_mask_path)
                        saved_count += 1
                    except Exception as e:
                        print(f"Verification failed for {base_name}: {e}")
                        out_img_path.unlink(missing_ok=True)
                        out_mask_path.unlink(missing_ok=True)
                        failed_count += 1
            else:
                failed_count += 1

        print(f"Successfully saved {saved_count} images from {split} split")
        print(f"Failed: {failed_count}")

    def _process_single_image(self, img_path, inst_path, base_name, out_img_dir, out_mask_dir):
        try:
            image = np.array(Image.open(img_path))
            instances = np.array(Image.open(inst_path))
            
            new_height = int(image.shape[0] * self.scale_factor)
            new_width = int(image.shape[1] * self.scale_factor)
            resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
            resized_instances = cv2.resize(instances, (new_width, new_height), interpolation=cv2.INTER_NEAREST)
            
            combined_mask = self._create_combined_mask(resized_instances)
            
            out_img = Image.fromarray(resized_image)
            out_img.save(out_img_dir / f"{base_name}.png", optimize=False)
            out_img.close()
            
            np.save(out_mask_dir / f"{base_name}.npy", combined_mask)
            
            return True
        except Exception as e:
            print(f"Error processing {base_name}: {e}")
            return False
    
    def _create_combined_mask(self, instances):
        """
        Output encoding:
        0                = background
        1000 + k         = human instance k
        2000 + k         = vehicle instance k
        """
        new_mask = np.zeros_like(instances, dtype=np.int32)

        human_instance_counter = 1
        vehicle_instance_counter = 1

        unique_ids = np.unique(instances)

        for inst_id in unique_ids:
            if inst_id < 1000:
                continue

            class_id = inst_id // 1000

            # Humans
            if class_id in {24, 25}:
                new_mask[instances == inst_id] = 1000 + human_instance_counter
                human_instance_counter += 1

            # Vehicles
            elif class_id in {26, 27, 28, 31, 32, 33}:
                new_mask[instances == inst_id] = 2000 + vehicle_instance_counter
                vehicle_instance_counter += 1

        return new_mask



print("Starting preprocessing...")
preprocessor = CityscapesPreprocessor(
    input_root='original_dataset/',
    output_root='processed_dataset/',
    scale_factor=0.25
)
preprocessor.process_all_splits()

print("\n" + "="*50)
print("Preprocessing complete!")
print("="*50)

Starting preprocessing...

Processing train split...


train: 100%|██████████| 2975/2975 [10:29<00:00,  4.73it/s]


Successfully saved 2975 images from train split
Failed: 0

Processing val split...


val: 100%|██████████| 500/500 [01:48<00:00,  4.63it/s]


Successfully saved 500 images from val split
Failed: 0

Processing test split...


test: 100%|██████████| 1525/1525 [05:15<00:00,  4.84it/s]

Successfully saved 1525 images from test split
Failed: 0

Preprocessing complete!
